# CHAPTER 5 - QAOA: Quantum Approximate Optimization Algorithm - Qiskit Code



*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install pylatexenc==2.10

In [ ]:
pip install qiskit-optimization==0.4.0

In [16]:
#from qiskit.opflow import Z
from qiskit.quantum_info import Pauli
#from qiskit.algorithms import QAOA # depreciado
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

#from utils.algorithm_globals import algorithm_globals, QuantumInstance
import numpy as np
# Setting our seed value
np.random.seed(1234)

from qiskit import Aer

#H1 = Z^Z # Define Z_0Z_1 # depreciado
H1 = Pauli('ZZ') # Define Z_0Z_1

#seed = 1234
#algorithm_globals.random_seed = seed
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), 
                   seed_simulator=seed, seed_transpiler=seed, 
                   shots = 10)
qaoa = QAOA(optimizer = COBYLA(), quantum_instance=quantum_instance)

#qaoa = QAOA() 3 depreciado
circuit = qaoa.construct_circuit([1,2],H1)[0]
circuit.draw(output="mpl")

ImportError: cannot import name 'Aer' from 'qiskit' (/home/furtado/anaconda3/envs/TesteqmlPraticalOpt/lib/python3.9/site-packages/qiskit/__init__.py)

In [ ]:
circuit.decompose().decompose().draw(output="mpl")


In [ ]:
qaoa = QAOA(reps = 2)
circuit = qaoa.construct_circuit([1,2,3,4],H1)[0]
circuit.decompose().decompose().draw(output="mpl")

In [ ]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA

seed = 1234
algorithm_globals.random_seed = seed
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), 
                   seed_simulator=seed, seed_transpiler=seed, 
                   shots = 10)
qaoa = QAOA(optimizer = COBYLA(), quantum_instance=quantum_instance)
result = qaoa.compute_minimum_eigenvalue(H1)
print(result)

*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
ibm_token="1234"

In [ ]:
from qiskit import IBMQ

IBMQ.save_account(ibm_token)
provider = IBMQ.load_account()
program_id = "qaoa"

H1 = Z^Z
opt = COBYLA()
reps = 1
shots = 1024

runtime_inputs = {
    "operator": H1,
    "reps": reps,
    "optimizer": opt,
    "initial_point": [0,0],
    "use_swap_strategies": False
}

options = {"backend_name": "ibmq_belem"}

job = provider.runtime.run(program_id=program_id, 
      options=options, inputs=runtime_inputs)

In [ ]:
result = job.result()
print("Optimizer time", result['optimizer_time'])
print("Optimal value", result['optimal_value'])
print("Optimal point", result['optimal_point'])
print("Optimal state", result['eigenstate'])

In [ ]:
from qiskit_optimization.problems import QuadraticProgram

qp = QuadraticProgram()
qp.binary_var('x')
qp.binary_var('y')
qp.binary_var('z')

qp.minimize(linear = {'y':-1}, quadratic = {('x','y'):2, ('z','y'):-4})
qp.linear_constraint(linear = {'x':1, 'y':2, 'z':3}, 
    sense ="<=", rhs = 5)

print(qp.export_as_lp_string())

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import NumPyMinimumEigensolver
np_solver = NumPyMinimumEigensolver()
np_optimizer = MinimumEigenOptimizer(np_solver)
result = np_optimizer.solve(qp)
print(result)

In [ ]:
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), 
    shots = 1024)
qaoa = QAOA(optimizer = COBYLA(), 
    quantum_instance=quantum_instance, reps = 1)  
qaoa_optimizer = MinimumEigenOptimizer(qaoa)
result = qaoa_optimizer.solve(qp)
print(result)

In [ ]:
print('Variable order:', [var.name for var in result.variables])
for s in result.samples:
    print(s)

In [ ]:
print(result.min_eigen_solver_result)


In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo
qp_to_qubo = QuadraticProgramToQubo()
qubo = qp_to_qubo.convert(qp)
print(qubo.export_as_lp_string())

In [ ]:
H1, offset = qubo.to_ising()
print("The Hamiltonian is", H1)
print("The constant term is", offset)

In [ ]:
from qiskit_optimization.runtime import QAOAClient
from qiskit import IBMQ

provider = IBMQ.load_account()
qaoa_client = QAOAClient(provider=provider, 
                      backend=provider.get_backend("ibm_oslo"), reps=1)

qaoa = MinimumEigenOptimizer(qaoa_client)
result = qaoa.solve(qp)
print(result)